# Dependency

In [ ]:
import numpy as np
import math
from sklearn import metrics
from sklearn.metrics import roc_curve
import tensorflow as tf
from tensorflow.keras import layers,Model
from sklearn.model_selection import KFold
import gc
import import_ATP as load_data
import MCNN

# PARAM

In [2]:
DATA_LABEL=load_data.data_label()
NUM_DEPENDENT =7
MAXSEQ = NUM_DEPENDENT*2+1

NUM_FEATURE = 1024
NUM_FILTER = 1024
NUM_HIDDEN = 1000
BATCH_SIZE  = 1024
WINDOW_SIZES = [4,6,8,10]
EPOCHS      = 20



NUM_CLASSES = 2
CLASS_NAMES = ['Negative','Positive']



K_Fold = 5
VALIDATION_MODE="independent"
#"independent" "cross"

# Main

In [ ]:
# Example usage:
x_train,y_train,x_test,y_test = load_data.MCNN_data_load(NUM_CLASSES,NUM_DEPENDENT)

In [11]:
def model_test(model, x_test, y_test):

    print(x_test.shape)
    pred_test = model.predict(x_test)
    fpr, tpr, thresholds = roc_curve(y_test[:,1], pred_test[:, 1])
    AUC = metrics.auc(fpr, tpr)
    #tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
    display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=AUC, estimator_name='mCNN')
    display.plot()
    

    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    # locate the index of the largest g-mean
    ix = np.argmax(gmeans)
    print(f'Best Threshold={thresholds[ix]}, G-Mean={gmeans[ix]}')
    threshold = thresholds[ix]

    y_pred = (pred_test[:, 1] >= threshold).astype(int)

    TN, FP, FN, TP =  metrics.confusion_matrix(y_test[0:][:,1], y_pred).ravel()

    Sens = TP/(TP+FN) if TP+FN > 0 else 0.0
    Spec = TN/(FP+TN) if FP+TN > 0 else 0.0
    Acc = (TP+TN)/(TP+FP+TN+FN)
    MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)) if TP+FP > 0 and FP+TN > 0 and TP+FN and TN+FN else 0.0
    F1 = 2*TP/(2*TP+FP+FN)
    Prec=TP/(TP+FP)
    Recall=TP/(TP+FN)
    return TP,FP,TN,FN,Sens,Spec,Acc,MCC,AUC

In [12]:
if(VALIDATION_MODE=="cross"):	
	kfold = KFold(n_splits = K_Fold, shuffle = True, random_state = 2)
	results=[]
	i=1
	for train_index, test_index in kfold.split(x_train):
		print(i,"/",K_Fold,'\n')
		# 取得訓練和測試數據
		X_train, X_test = x_train[train_index], x_train[test_index]
		Y_train, Y_test = y_train[train_index], y_train[test_index]
		print(X_train.shape)
		print(X_test.shape)
		print(Y_train.shape)
		print(Y_test.shape)
		# 重新建模
		model = MCNN.DeepScan(
    		num_filters=NUM_FILTER,
			num_hidden=NUM_HIDDEN,
			window_sizes=WINDOW_SIZES)
		model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
		model.build(input_shape=X_train.shape)
		# 在測試數據上評估模型
		history=model.fit(
			X_train,
			Y_train,
			batch_size=BATCH_SIZE,
			epochs=EPOCHS,
			callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)],
			verbose=1,
			shuffle=True
		)
		TP, FP, TN, FN, Sens, Spec, Acc, MCC, AUC = model_test(model, X_test, Y_test)
		results.append([TP, FP, TN, FN, Sens, Spec, Acc, MCC, AUC])
		i+=1
		
		del X_train
		del X_test
		del Y_train
		del Y_test
		gc.collect()
		
	mean_results = np.mean(results, axis=0)
	print(f'TP={mean_results[0]:.4}, FP={mean_results[1]:.4}, TN={mean_results[2]:.4}, FN={mean_results[3]:.4}, Sens={mean_results[4]:.4}, Spec={mean_results[5]:.4}, Acc={mean_results[6]:.4}, MCC={mean_results[7]:.4}, AUC={mean_results[8]:.4}\n')


In [ ]:
if(VALIDATION_MODE=="independent"):
	model = MCNN.DeepScan(
		num_filters=NUM_FILTER,
		num_hidden=NUM_HIDDEN,
		window_sizes=WINDOW_SIZES)
	model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
	model.build(input_shape=x_train.shape)
	model.summary()

	model.fit(
		x_train,
		y_train,
		batch_size=BATCH_SIZE,
		epochs=EPOCHS,
		shuffle=True,
	)
	TP,FP,TN,FN,Sens,Spec,Acc,MCC,AUC,display = model_test(model, x_test, y_test)
	print(f'TP={mean_results[0]:.4}, FP={mean_results[1]:.4}, TN={mean_results[2]:.4}, FN={mean_results[3]:.4}, Sens={mean_results[4]:.4}, Spec={mean_results[5]:.4}, Acc={mean_results[6]:.4}, MCC={mean_results[7]:.4}, AUC={mean_results[8]:.4}\n')
	display.plot()